## Эмулятор машины Тьюринга

Описание машины Тьюринга должно быть в файле, формат следующий:

first_state first_sym second_sym direction second_state

<текущее состояние> <текущий символ> <новый символ> <направление> <новое состояние>

<текущее состояние> <текущий символ> <новый символ> <направление> <новое состояние>

...

Обозначения:

Начальное состояние должно называться "0"

Конечное состояние "halt" (или "halt-accept", "halt-reject")

Символы могут быть любыми, но "_" - это пустой символ, "*" - это обозначение любого символа

Направление: "l" = влево, "r" = вправо, "*" = стоять на месте

In [1]:
import pandas as pd

In [2]:
class step():
    def __init__(self, string, position, state):
        self._string = string
        self._position = position
        self._state = state
        
    @property
    def string(self):
        return self._string
    @string.setter
    def string(self, value):
        self._string = value
    
    def __str__(self):
        i=0
        while i<len(self.string) and self.string[i]=='_':
            i += 1
        j=len(self.string)-1
        while j>=0 and self.string[j]=='_':
            j -= 1
        if i>=j+1:
            return '_'
        return self.string[i:(j+1)]
    
    @property
    def position(self):
        return self._position
    @position.setter
    def position(self, value):
        self._position = value
    
    @property
    def state(self):
        return self._state
    @state.setter
    def state(self, value):
        self._state = value
    
    @property
    def symbol(self):
        return self.string[self.position]
    
    def change_string(self, next_sym, new_position):
        pos = self.position
        if new_position==-1:
            self.string = '_' + next_sym + self.string[(pos+1):]
            self.position = 0
        elif new_position==len(self.string):
            self.string = self.string[:pos] + next_sym + '_'
            self.position = new_position
        else:
            self.string = self.string[:pos] + next_sym + self.string[(pos+1):]
            self.position = new_position

    @property
    def check_state(self):
        if self.state=='halt' or self.state=='halt-accept' or self.state=='halt-reject':
            return 0
        return 1

class Turing_Machine():
    def __init__(self, path, string = '_'):
        self.states = pd.read_csv(path, sep=' ')
        self.current = step(string, 0, '0')
        
    def do_step(self):
        sym = self.current.symbol
        
        new = self.states[(self.states['first_state']==self.current.state) & (self.states['first_sym']==sym)]
        if new.size==0:
            new = self.states[(self.states['first_state']==self.current.state) & (self.states['first_sym']=='*')]
        
        self.current.state = new['second_state'].ravel()[0]
        
        direct = new['direction'].ravel()[0]
        new_position = self.current.position
        if direct=='l':
            new_position -= 1
        elif direct=='r':
            new_position += 1
        
        next_sym = new['second_sym'].ravel()[0] if new['second_sym'].ravel()[0]!='*' else sym
        self.current.change_string(next_sym, new_position)
        
    def execution(self):
        while self.current.check_state:
            self.do_step()
        print(self.current)
    
    def execution_print(self):
        while self.current.check_state:
            self.do_step()
            print(self.current)
        print 'The End!'
    
    def set_string(self, string, pos = 0):
        self.current.string = string
        self.current.state = '0'
        self.current.position = pos
    
    def one_step(self):
        if self.current.check_state:
            self.do_step()
        print(self.current)
        if self.current.check_state==0:
            print 'The End!'
            return 0
        return 1

In [3]:
path1 = 'palindrome.txt'
path2 = 'division.txt'

string1 = '1001001001'
string2 = '1101011'
string3 = '111011'

string4 = '11111011'
string5 = '1101'

Создаем 2 машины:

In [4]:
check_palindrome = Turing_Machine(path1, string1)
# Проверка числа из 0 и 1 на палиндром

unary_division = Turing_Machine(path2, string4)
# Унарное деление, 0 - разделитель между делимым и делителем, в ответе 0 разделитель между частным и остатком

Обычный запуск:

In [5]:
check_palindrome.execution()

:)


In [6]:
unary_division.execution()

1101


Меняем начальную строку:

In [7]:
check_palindrome.set_string(string2)
unary_division.set_string(string5)

Печатаем все промежуточные шаги:

In [8]:
check_palindrome.execution_print()

101011
101011
101011
101011
101011
101011
101011
101011
10101
10101
10101
10101
10101
10101
10101
0101
0101
0101
0101
0101
0101
010
010
010
010
010
10
10
10
10
1
1
1
_
_
_
:
:)
The End!


In [9]:
unary_division.execution_print()

1101
1101
1101
1101
110:
110:
1:0:
1:0:
1:0:
1:0:
1:0:_1
1:0:_1
1:01_1
1:01_1
1%01_1
1%01_1
1%01_1
1%01_1
1%0:_1
1%0:_1
1%0:_1
:%0:_1
:%0:_1
:%0:_1
:%0:_1
:%0:_1
:%0:_1
:%0:_11
:%0:_11
:%0:_11
:%01_11
:%01_11
:%01_11
%%01_11
%%01_11
%01_11
01_11
1_11
11
11
The End!


Для выполнения по шагам:

In [10]:
check_palindrome.set_string(string3)

In [11]:
check_palindrome.one_step()

11011


1

In [12]:
check_palindrome.one_step()

11011


1

In [28]:
while check_palindrome.one_step():
    pass

11011
11011
11011
11011
11011
1101
1101
1101
1101
1101
1101
101
101
101
101
101
10
10
10
10
0
0
0
0
_
:
:(
The End!
